In [ ]:
from glob import glob
import keras
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import model_from_json

def chunk(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

def get_id_from_file_path(file_path):
    return file_path.split(os.path.sep)[-1].replace('.tif', '')

test_files = glob('../test/*.tif')

model = model_from_json('model.json')
model.load_weights(glob('./ResNeXt*.h5')[-1])

predicts = []
ids = []
batch_size = 32

for itr,batch in enumerate(chunk(test_files, batch_size)):
    print('\rPrograss : %d/%d'%(itr*batch_size,len(test_files)))
    X = [(cv2.imread(x)/255) for x in batch]
    ids_batch = [get_id_from_file_path(x) for x in batch]
    X = np.array(X)
    preds_batch = ((
        model.predict(X).ravel()*model.predict(X[:, ::-1, :, :]).ravel()*model.predict(X[:, ::-1, ::-1, :]).ravel()*model.predict(X[:, :, ::-1, :]).ravel())**0.25).tolist()
    preds += preds_batch
    ids += ids_batch
    
df = pd.DataFrame({'id':ids, 'label':preds})
df.to_csv("resnext.csv", index=False)
df.head()
